In [43]:
import pandas as pd
import numpy as np
from numpy import asarray

In [44]:
from PIL import Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import os
from os import listdir

In [45]:
damagepath = "/code/Project2/COE379L-Project2/damage"
nodamagepath = "/code/Project2/COE379L-Project2/no_damage"

# Create data as a dictionary
data = []

for images in os.listdir(damagepath):
    if (images.endswith(".png") or images.endswith(".jpg")\
        or images.endswith(".jpeg")):
        img_path = os.path.join(damagepath, images)
        img = load_img(img_path, target_size=(128,128))
        imagedata = img_to_array(img) / 255.0
        data.append({"Image": imagedata, "Status": 1})

for images in os.listdir(nodamagepath):
    if (images.endswith(".png") or images.endswith(".jpg")\
        or images.endswith(".jpeg")):
        img_path = os.path.join(damagepath, images)
        img = load_img(img_path, target_size=(128,128))
        imagedata = img_to_array(img) / 255.0
        data.append({"Image": imagedata, "Status": 0})

# I want to store the data as numpy arrays, since the images are colored => 3D array
data = pd.DataFrame(data)

FileNotFoundError: [Errno 2] No such file or directory: '/code/Project2/COE379L-Project2/damage/-95.079149_30.031708000000002.jpeg'

In [ ]:
# investigate data for engineering
data.head()

In [ ]:
data.info()